# **Cointab Software Private Limited**

# *Data Analyst - Assignment*

# Import Libraries and read **files**

In [144]:
import math
import numpy as np
import pandas as pd
order_report = pd.read_excel("/content/Company X - Order Report.xlsx")
pin_zone = pd.read_excel("/content/Company X - Pincode Zones.xlsx")
sku = pd.read_excel("/content/Company X - SKU Master.xlsx")
invoice = pd.read_excel("/content/Courier Company - Invoice.xlsx")
rates = pd.read_excel("/content/Courier Company - Rates.xlsx")

# **Missing Values**

In [145]:
print("Missing Values in order Report are :\n",order_report.isna().sum())
print("Missing Values in Pincode are :\n",pin_zone.isna().sum())
print("Missing Values in SKU are :\n",sku.isna().sum())
print("Missing Values in Invoice are :\n",invoice.isna().sum())
print("Missing Values in Rates are :\n",rates.isna().sum())

Missing Values in order Report are :
 ExternOrderNo    0
SKU              0
Order Qty        0
dtype: int64
Missing Values in Pincode are :
 Warehouse Pincode    0
Customer Pincode     0
Zone                 0
dtype: int64
Missing Values in SKU are :
 SKU           0
Weight (g)    0
dtype: int64
Missing Values in Invoice are :
 AWB Code                0
Order ID                0
Charged Weight          0
Warehouse Pincode       0
Customer Pincode        0
Zone                    0
Type of Shipment        0
Billing Amount (Rs.)    0
dtype: int64
Missing Values in Rates are :
 Zone                                     0
Weight Slabs                             0
Forward Fixed Charge                     0
Forward Additional Weight Slab Charge    0
RTO Fixed Charge                         0
RTO Additional Weight Slab Charge        0
dtype: int64


#Duplicate Value

In [146]:
print("Number of duplicate rows in order Report is :",order_report.duplicated().sum())
print("Number of duplicate rows in Pincode is :",pin_zone.duplicated().sum())
print("Number of duplicate rows in SKU is :",sku.duplicated().sum())
print("Number of duplicate rows in Invoice is :",invoice.duplicated().sum())
print("Number of duplicate rows in Rates is :",rates.duplicated().sum())

Number of duplicate rows in order Report is : 2
Number of duplicate rows in Pincode is : 16
Number of duplicate rows in SKU is : 1
Number of duplicate rows in Invoice is : 0
Number of duplicate rows in Rates is : 0


# **Drop** Duplicates

In [147]:
# Dropping duplicate rows
order_report.drop_duplicates(inplace=True)
sku.drop_duplicates(inplace=True)
pin_zone.drop_duplicates(inplace=True)

#Merge

In [148]:
# Add weight per item column to order report
data = order_report.merge(sku,how='left',on='SKU')
print(data)

     ExternOrderNo            SKU  Order Qty  Weight (g)
0       2001827036  8904223818706        1.0         127
1       2001827036  8904223819093        1.0         150
2       2001827036  8904223819109        1.0         100
3       2001827036  8904223818430        1.0         165
4       2001827036  8904223819277        1.0         350
..             ...            ...        ...         ...
393     2001806229  8904223818942        1.0         133
394     2001806229  8904223818850        1.0         240
395     2001806226  8904223818850        2.0         240
396     2001806210  8904223816214        1.0         120
397     2001806210  8904223818874        1.0         100

[398 rows x 4 columns]


# total weight of item in KG

In [149]:
data['Total_Weight_by_X(kg)'] = data['Weight (g)']*data['Order Qty']/1000
print(data)

     ExternOrderNo            SKU  Order Qty  Weight (g)  \
0       2001827036  8904223818706        1.0         127   
1       2001827036  8904223819093        1.0         150   
2       2001827036  8904223819109        1.0         100   
3       2001827036  8904223818430        1.0         165   
4       2001827036  8904223819277        1.0         350   
..             ...            ...        ...         ...   
393     2001806229  8904223818942        1.0         133   
394     2001806229  8904223818850        1.0         240   
395     2001806226  8904223818850        2.0         240   
396     2001806210  8904223816214        1.0         120   
397     2001806210  8904223818874        1.0         100   

     Total_Weight_by_X(kg)  
0                    0.127  
1                    0.150  
2                    0.100  
3                    0.165  
4                    0.350  
..                     ...  
393                  0.133  
394                  0.240  
395               

# Drop unnecessary columns

In [150]:

data.drop(labels=['Order Qty','Weight (g)','SKU'],axis=1,inplace=True)
print(data)

     ExternOrderNo  Total_Weight_by_X(kg)
0       2001827036                  0.127
1       2001827036                  0.150
2       2001827036                  0.100
3       2001827036                  0.165
4       2001827036                  0.350
..             ...                    ...
393     2001806229                  0.133
394     2001806229                  0.240
395     2001806226                  0.480
396     2001806210                  0.120
397     2001806210                  0.100

[398 rows x 2 columns]


# Total weight and Amount for each order

In [151]:

data = data.groupby(by=['ExternOrderNo'],as_index=False).sum()
data.shape

(124, 2)

# Lets merge invoice data

In [152]:

data = data.merge(invoice,how='left',left_on='ExternOrderNo',right_on='Order ID')
print(data)


     ExternOrderNo  Total_Weight_by_X(kg)       AWB Code    Order ID  \
0       2001806210                  0.220  1091117221940  2001806210   
1       2001806226                  0.480  1091117222065  2001806226   
2       2001806229                  0.500  1091117222080  2001806229   
3       2001806232                  1.302  1091117222124  2001806232   
4       2001806233                  0.245  1091117222135  2001806233   
..             ...                    ...            ...         ...   
119     2001821995                  0.477  1091121183730  2001821995   
120     2001822466                  1.376  1091121305541  2001822466   
121     2001823564                  0.672  1091121666133  2001823564   
122     2001825261                  1.557  1091121981575  2001825261   
123     2001827036                  1.676  1091122418320  2001827036   

     Charged Weight  Warehouse Pincode  Customer Pincode Zone  \
0              2.92             121003            140604    b   
1    

# merge invoice with pincode data
# rename zone columns from invoice data as zone_by_courier and pincode as zone_by_X

In [153]:

data = data.merge(pin_zone,how='left',on=['Warehouse Pincode','Customer Pincode'])


data = data.rename(columns={'Zone_x':'Zone_by_Courier','Zone_y':'Zone_by_X'})

In [154]:
rates['Zone'].unique()

array(['A', 'B', 'C', 'D', 'E'], dtype=object)

# Convert Uppercase to lowercase

In [155]:

rates['Zone'] = rates['Zone'].str.lower()

In [156]:
data = data.merge(rates,how='left',left_on='Zone_by_X',right_on='Zone')

# Forward additional charge

In [157]:

data['Forward_additional_charge'] = (data['Total_Weight_by_X(kg)']/data['Weight Slabs']).astype(int)*data['Forward Additional Weight Slab Charge']

# Fixed RTO charges for forward and rto order

In [158]:

def fixed_rto_charge(row):
    if row['Type of Shipment'] == 'Forward and RTO charges':
        return row['RTO Fixed Charge']
    elif row['Type of Shipment'] == 'Forward charges':
        return 0
    else:
        return None

data['RTO_fixed_charge'] = data.apply(fixed_rto_charge, axis=1)

# Additional RTO charges for forward and rto order

In [159]:

def additional_rto_charge(row):
    if row['Type of Shipment'] == 'Forward and RTO charges':
        return math.floor(row['Total_Weight_by_X(kg)']/row['Weight Slabs'])*row['RTO Additional Weight Slab Charge']
    elif row['Type of Shipment'] == 'Forward charges':
        return 0
    else:
        return None

data['RTO_additional_charge'] = data.apply(additional_rto_charge, axis=1)

# Calculate total expected charge for delivery

In [160]:

data['Expected_charges'] = data['Forward Fixed Charge'] + data['Forward_additional_charge'] + data['RTO_fixed_charge'] + data['RTO_additional_charge']

# Difference between expected and billed charges

In [161]:

data['diff_charges'] =  data['Expected_charges'] - data['Billing Amount (Rs.)']

# Add column weight slab charged by courier for output

In [162]:


data['Weight_slabs_Courier'] = data['Zone_by_Courier'].replace(to_replace=['a','b','c','d','e'],value=[0.25,0.5,0.75,1.25,1.5])

# add column to check if courier company overcharged or undercharged

In [163]:

def is_correct(row):
    if row['diff_charges'] == 0:
        return 'X correctly charged'
    elif row['diff_charges'] > 0:
        return 'X Undercharged'
    else:
        return 'X Overcharged'
data['over_or_under_charged'] = data.apply(is_correct,axis=1)

In [164]:
data.columns

Index(['ExternOrderNo', 'Total_Weight_by_X(kg)', 'AWB Code', 'Order ID',
       'Charged Weight', 'Warehouse Pincode', 'Customer Pincode',
       'Zone_by_Courier', 'Type of Shipment', 'Billing Amount (Rs.)',
       'Zone_by_X', 'Zone', 'Weight Slabs', 'Forward Fixed Charge',
       'Forward Additional Weight Slab Charge', 'RTO Fixed Charge',
       'RTO Additional Weight Slab Charge', 'Forward_additional_charge',
       'RTO_fixed_charge', 'RTO_additional_charge', 'Expected_charges',
       'diff_charges', 'Weight_slabs_Courier', 'over_or_under_charged'],
      dtype='object')

In [165]:
output = data[['Order ID','AWB Code','Total_Weight_by_X(kg)','Weight Slabs','Charged Weight','Weight_slabs_Courier','Zone_by_X',
               'Zone_by_Courier','Expected_charges','Billing Amount (Rs.)','diff_charges']]
print(output)

       Order ID       AWB Code  Total_Weight_by_X(kg)  Weight Slabs  \
0    2001806210  1091117221940                  0.220           1.0   
1    2001806226  1091117222065                  0.480           1.5   
2    2001806229  1091117222080                  0.500           1.5   
3    2001806232  1091117222124                  1.302           1.5   
4    2001806233  1091117222135                  0.245           1.0   
..          ...            ...                    ...           ...   
119  2001821995  1091121183730                  0.477           1.0   
120  2001822466  1091121305541                  1.376           1.0   
121  2001823564  1091121666133                  0.672           1.5   
122  2001825261  1091121981575                  1.557           1.5   
123  2001827036  1091122418320                  1.676           2.0   

     Charged Weight  Weight_slabs_Courier Zone_by_X Zone_by_Courier  \
0              2.92                  0.50         b               b   
1    

In [167]:
summary = pd.DataFrame(index=['Total orders where X has been correctly charged'
                              ,'Total Orders where X has been overcharged'
                              ,'Total Orders where X has been undercharged'
                             ], columns=['Count','Amount(Rs.)'])


summary['Count'] = [(data['diff_charges']==0).sum(),
                    (data['diff_charges']<0).sum(),
                    (data['diff_charges']>0).sum()]

summary['Amount(Rs.)'] = [data[data['diff_charges']==0]['Billing Amount (Rs.)'].sum()
                          ,abs(data[data['diff_charges']<0]['diff_charges'].sum())
                          ,data[data['diff_charges']>0]['diff_charges'].sum()]
print(summary)

                                                 Count  Amount(Rs.)
Total orders where X has been correctly charged      7        321.3
Total Orders where X has been overcharged          114       7751.2
Total Orders where X has been undercharged           3         47.2


In [168]:
output.to_excel("/content/Output.xlsx")
summary.to_excel("/content/Summary.xlsx")